In [1]:
import numpy as np
import pandas as pd
import keras
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from gensim.models.doc2vec import LabeledSentence
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
import keras.layers.merge
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras.callbacks import EarlyStopping
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import codecs
import matplotlib.pyplot as plt
from gensim.models import Doc2Vec
import pickle

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
EMBEDDING_DIM = 700 # how big is each word vector
MAX_VOCAB_SIZE = 175303 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 80 # max number of words in a comment to use

#training params
batch_size = 256  
num_epochs = 20 

In [3]:
def sentiment_label(polarity):
    if polarity=='negative':
        return 0
    else:
        return 1

In [4]:
test_set = pd.read_csv('./corpus/tripadvisor/test.csv')
test_set['sentiment'] = test_set['polarity'].apply(sentiment_label)
test_set.head()

,content,polarity,sentiment
0,lokasi hotel tidak jauh dari komplek mall kali...,negative,0
1,lokasi hotel tidak jauh dengan taksi ke area m...,negative,0
2,lokasi hotel yang sangat strategis masih di ko...,negative,0
3,lokasi hotel yang sangat strategis masih di ko...,negative,0
4,lokasi hotel yang strategis di tengah kota dan...,negative,0


In [5]:
from sklearn.cross_validation import train_test_split
SEED = 2000

x_train, x_validation, y_train, y_validation = train_test_split(test_set['content'], test_set['sentiment'], test_size=.1, random_state=SEED)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
# tokenizer = RegexpTokenizer(r'\w+')
# clean_train_comments = pd.read_csv("./corpus/tripadvisor/train_set.csv")
# clean_train_comments['content'] = clean_train_comments['content'].astype('str') 
# clean_train_comments.dtypes
# clean_train_comments["tokens"] = clean_train_comments["content"].apply(tokenizer.tokenize)
# clean_train_comments['sentiment'] = clean_train_comments['polarity'].apply(sentiment_label)
   
# clean_train_comments.head()

In [7]:
# clean_test_comments = pd.read_csv("./corpus/tripadvisor/test_set.csv")
# clean_test_comments['content'] = clean_test_comments['content'].astype('str') 
# clean_test_comments.dtypes
# clean_test_comments["tokens"] = clean_test_comments["content"].apply(tokenizer.tokenize)
# clean_test_comments['sentiment'] = clean_test_comments['polarity'].apply(sentiment_label)

# clean_test_comments.head()

In [8]:
# all_training_words = [word for tokens in clean_train_comments["tokens"] for word in tokens]
# training_sentence_lengths = [len(tokens) for tokens in clean_train_comments["tokens"]]
# TRAINING_VOCAB = sorted(list(set(all_training_words)))
# print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
# print("Max sentence length is %s" % max(training_sentence_lengths))

In [9]:
# all_test_words = [word for tokens in clean_test_comments["tokens"] for word in tokens]
# test_sentence_lengths = [len(tokens) for tokens in clean_test_comments["tokens"]]
# TEST_VOCAB = sorted(list(set(all_test_words)))
# print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
# print("Max sentence length is %s" % max(test_sentence_lengths))

In [10]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
# x_train = labelize_text(clean_train_comments["content"], 'TRAIN')
# x_validation = labelize_text(clean_test_comments["content"], 'TEST')

x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [11]:
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
# word2vec = Word2Vec.load('./vectorizer/prosa/word2vec.model')

In [12]:
tfidf = pickle.load(open('./vectorizer/tripadvisor/tfidf.pickle', 'rb'))
model_dbow = Doc2Vec.load("./vectorizer/tripadvisor/model_dbow.model")
model_dmc = Doc2Vec.load("./vectorizer/tripadvisor/model_dmc.model")
model_dmm = Doc2Vec.load("./vectorizer/tripadvisor/model_dmm.model")

In [13]:
def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmm[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

In [14]:
def build_Vector(tokens, word_size, doc_size):
    doc_vec = build_doc_Vector(tokens, doc_size)
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), doc_size + word_size))
    for word in tokens:
        try:
            word_vec = np.append(doc_vec, word2vec[word])
            vec = np.append(vec, word_vec)
        except KeyError: 
            word_vec = np.append(doc_vec, np.zeros((1, word_size)))
            vec = np.append(vec, word_vec)
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, doc_size + word_size)
    return vec

In [15]:
train_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
746it [00:03, 207.13it/s]
83it [00:00, 242.70it/s]


In [16]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))
val_vecs = val_vecs.reshape((num_data_val, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM))

In [17]:
def ConvNet(max_sequence_length, embedding_dim, labels_index, trainable=False, extra_conv=True):

    sequence_input = Input(shape=(max_sequence_length, embedding_dim,), dtype='float32')

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(sequence_input)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    #l_merge = Merge(mode='concat', concat_axis=1)(convs)
    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(sequence_input)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc']) 
    model.summary()
    return model

In [18]:
# y_tr = clean_train_comments['sentiment'].values
# y_ts = clean_test_comments['sentiment'].values
y_tr = y_train.values
y_ts = y_validation.values

In [19]:
x_train = train_vecs
y_train = y_tr

x_test = val_vecs
y_test = y_ts

In [20]:
model = ConvNet(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM, 1, False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 80, 700)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 78, 128)      268928      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 77, 128)      358528      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 76, 128)      448128      input_1[0][0]                    
__________________________________________________________________________________________________
max_poolin

In [21]:
hist = model.fit(x_train, y_train, epochs=num_epochs, validation_data=(x_test, y_test), batch_size=batch_size)

Train on 746 samples, validate on 83 samples
Epoch 1/20
746/746 [==============================] - ETA: 8s - loss: 1.5880 - acc: 0.503 - ETA: 3s - loss: 4.0171 - acc: 0.546 - 11s 14ms/step - loss: 4.6708 - acc: 0.5684 - val_loss: 5.3506 - val_acc: 0.6627
Epoch 2/20
746/746 [==============================] - ETA: 5s - loss: 6.1626 - acc: 0.589 - ETA: 2s - loss: 5.8674 - acc: 0.599 - 10s 13ms/step - loss: 5.6746 - acc: 0.6019 - val_loss: 3.0616 - val_acc: 0.6867
Epoch 3/20
746/746 [==============================] - ETA: 6s - loss: 3.1399 - acc: 0.675 - ETA: 2s - loss: 2.2102 - acc: 0.664 - 10s 13ms/step - loss: 1.7767 - acc: 0.6568 - val_loss: 0.8713 - val_acc: 0.4578
Epoch 4/20
746/746 [==============================] - ETA: 6s - loss: 1.1551 - acc: 0.554 - ETA: 2s - loss: 1.0757 - acc: 0.576 - 10s 13ms/step - loss: 0.9890 - acc: 0.5898 - val_loss: 0.4444 - val_acc: 0.7470
Epoch 5/20
746/746 [==============================] - ETA: 5s - loss: 0.5203 - acc: 0.777 - ETA: 2s - loss: 0.4639 

In [22]:
# model.save('./model/yoon_kim_pv/cnn_model_04.h5') 

In [23]:
# model = load_model('./model/yoon_kim_pv/cnn_model_04.h5')

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_predict = model.predict(x_test, batch_size=256, verbose=1)
for i in range(len(y_predict)):
    y_predict[i][0] = round(y_predict[i][0])
print(classification_report(y_test, y_predict, labels = [0, 1], digits=8))

Test loss: 0.3189003338296729
Test accuracy: 0.975903619484729
83/83 [==============================] - 1s 7ms/step
             precision    recall  f1-score   support

          0  0.96491228 1.00000000 0.98214286        55
          1  1.00000000 0.92857143 0.96296296        28

avg / total  0.97674910 0.97590361 0.97567253        83

